In [48]:
###basic csv file creation
import csv

with open('persons.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Profession'])
    writer.writerow(['Derek', 'Software Developer'])
    writer.writerow(['Steve', 'Software Developer'])
    writer.writerow(['Paul', 'Manager'])

In [15]:
### writing multiple rows from a python list
import csv
row_list = [["SN", "Name", "Contribution"],
            [1, "Linus Torvalds", "Linux Kernel"],
            [2, "Tim Berners-Lee", "World Wide Web"],
            [3, "Guido van Rossum", "Python Programming"]]
with open('protagonist.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_list)

In [ ]:
### importing midi
import mido ###why doesn't this effing work?
from mido import MidiFile
mid = MidiFile('anthem.mid')
test_string = 'note_on'

### printing midi data;;; be advised that in MIDI format a note_on messsage with a velocity of 0 is equivalent to a note_off message
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(str(msg))

#how to see how many tracks in a MidiFile
mid.print_tracks(meta_only=True)

# mido.ticks2second(3840,mid.ticks_per_beat,)

In [17]:
### Getting only note_on messages
from mido import MidiFile
mid = MidiFile('ockIntroitus.mid')

def test(example):
    for i, track in enumerate(example.tracks):
        print('Track {}: {}'.format(i, track.name))
        for msg in track:
            if msg.type == 'note_on':
                if msg.velocity == 0:
                    print('note_off ' + str(msg)[8:]) # this is for MIDI tracks that implement note_offs as zero velocity note_ons ; standardizes to using note_offs
                else: print(str(msg))



In [22]:
### \\\ --- tools to generate DynamicNodeList from a midi file --- /// ###

from mido import MidiFile
import itertools
import csv
mid = MidiFile('test1.mid')
test_string = 'note_on'

def midi2list(midi_file): # gets only the note_on and note_off messages, creates a sublist for each, and breaks their parameters into separate strings
    for i, track in enumerate(midi_file.tracks):
        extraction = [ list(str(msg).split(" ")) for msg in track if msg.type == 'note_on' or msg.type == 'note_off' ]
        yield [ 'Track {} {}'.format(i, track.name), list(extraction) ]

step1 = list(midi2list(mid))
#print(step1)

def get_full(something): # filters out empty message lists (usually the meta message list is stored in track 0, so it's empty from the midi2list function filtering only note_on/note_off messages)
    return [ i for i in something if len(i[1]) != 0]

step2 = get_full(step1)
# print(step2)

def numberer(message_list): # adds number to beginning of each sub-list, which will be used in .csv creation to indicate row number (turns out this is not needed, I misunderstood how csv writer worked...)
    for entry in message_list:
        counter = 1
        for message in entry[1]:
            message.insert(0, counter)
            counter += 1
    return message_list

step3 = numberer(step2)
# print(step3)

def standardizer(numbed_list): # tests for note_on messages with velocity = 0; changes them to note_offs
    for entry in numbed_list:
        for message in entry[1]:
            if message[4] == 'velocity=0':
                 message[1] = 'note_off'
    return numbed_list

step4 = standardizer(step3)
# print(step4)

combined_func = standardizer(numberer(get_full(list(midi2list(mid)))))
# print(combined_func)

def abs_onsets(pl): # gets just the onset delta-times and converts them to an int list of absolute times
    for entry in pl:
        deltas = [ int( message[5].partition('=')[2]) for message in entry[1] ]
        yield itertools.accumulate(deltas)

onsetz = list(abs_onsets(combined_func))

def lister(blah): # this is just dealing with the yield iterator
    return [ list(item) for item in blah  ]

list_onsetz = lister(onsetz)
# print(list_onsetz)

def final_format(combined_func, onsets): # finally putting everything together
    for i, entry in enumerate(combined_func):
        for j in range(0,len(entry[1]), 2):
            yield [ onsets[i][j], onsets[i][j+1], combined_func[i][1][j][3].partition('=')[2], 'FALSE', 'FALSE',  onsets[i][j+1] - onsets[i][j] ]

partition_sizes = [len(entry[1]) // 2 for entry in combined_func] # but the yield doesn't distinguish between tracks, so we have to make this list for how many messages each track contains
# print(sizes)

ff = final_format(combined_func, list_onsetz)
# print(ff)

partitioned_list = [list(itertools.islice(ff, elem)) for elem in partition_sizes] #islice takes a generator for its first argument! i.e. the yield from 'final_format'. And we use 'partition_sizes' to tell islice how to group messages from the yield generator.
# print(partitioned_list)

# for entry in partitioned_list:
#     print(partitioned_list.index(entry))

def csv_maker(partitioned_list):
    for entry in partitioned_list:
        entry.insert(0, ['onset', 'terminus', 'vertex.id', 'onset.censored', 'terminus.censored', 'duration'])
        title = partitioned_list.index(entry) # titling each .csv file by its index ('0.csv' = track 1, etc.)
        with open('{}.csv'.format(title), 'w', newline='') as file: # 
            writer = csv.writer(file)
            writer.writerows(entry)

csv_maker(partitioned_list)
        

In [ ]:
###Pseudo code to be done:
### format to be passed to csv creator is:
    # [["Onset", "Terminus", "tail", "head"]]

#1. Make a dynamic node list in format:
    # [["Onset", "Terminus", "vertex.id", "onset.censored", "terminus.censored", "duration"]]
### Get list of all tempo changes
### For all tracks:
        i = 0
    # for all msg in track:
    
        if msg is a note_on messsage:
            create entry [i += 1,  ]

In [8]:
# lakjsdf = 'it=120'
# lakjsdf.partition('=')[2]

# test = [ 1, 2, 3, 4, 5]
# test[3:]

for i in range(0,10,2):
    print(i)

0
2
4
6
8
